# Bottom-Up Cube (BUC) Algorithm

In [15]:
%pip install memory_profiler

In [16]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from itertools import combinations
from memory_profiler import profile

In [33]:
data = pd.read_csv('Electric_Vehicle_Data.csv')
data.head(5)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,WAUTPBFF4H,King,Seattle,WA,98126.0,2017,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,34.0,235085336,POINT (-122.374105 47.54468),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,WAUUPBFF2J,Thurston,Olympia,WA,98502.0,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,22.0,237896795,POINT (-122.943445 47.059252),PUGET SOUND ENERGY INC,5.306701e+10
2,5YJSA1E22H,Thurston,Lacey,WA,98516.0,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,22.0,154498865,POINT (-122.78083 47.083975),PUGET SOUND ENERGY INC,5.306701e+10
3,1C4JJXP62M,Thurston,Tenino,WA,98589.0,2021,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,20.0,154525493,POINT (-122.85403 46.856085),PUGET SOUND ENERGY INC,5.306701e+10
4,5YJ3E1EC9L,Yakima,Yakima,WA,98902.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,14.0,225996361,POINT (-120.524012 46.5973939),PACIFICORP,5.307700e+10


In [34]:
print(len(data))

181458


In [35]:
def clean_df(df):
  remove_columns = ['VIN (1-10)','State','Postal Code','Electric Range','Base MSRP', 'Legislative District', 'DOL Vehicle ID','Vehicle Location', '2020 Census Tract']
  df = df.drop(columns=remove_columns)
  df = df.dropna()
  df = df.reset_index(drop=True)
  return df

In [36]:
columns = data.columns
print('Original columns:',columns)
data = clean_df(data)
print('New columns:', data.columns)

Original columns: Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract'],
      dtype='object')
New columns: Index(['County', 'City', 'Model Year', 'Make', 'Model',
       'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility',
       'Electric Utility'],
      dtype='object')


In [20]:
print(data.head(10))

      County               City  Model Year       Make     Model  \
0       King            Seattle        2017       AUDI        A3   
1   Thurston            Olympia        2018       AUDI        A3   
2   Thurston              Lacey        2017      TESLA   MODEL S   
3   Thurston             Tenino        2021       JEEP  WRANGLER   
4     Yakima             Yakima        2020      TESLA   MODEL 3   
5   Thurston            Olympia        2023       JEEP  WRANGLER   
6     Kitsap            Keyport        2017  CHEVROLET      VOLT   
7  Snohomish  Mountlake Terrace        2020      TESLA   MODEL 3   
8       King            Seattle        2022       AUDI        Q5   
9   Thurston            Olympia        2017  CHEVROLET      VOLT   

                    Electric Vehicle Type  \
0  Plug-in Hybrid Electric Vehicle (PHEV)   
1  Plug-in Hybrid Electric Vehicle (PHEV)   
2          Battery Electric Vehicle (BEV)   
3  Plug-in Hybrid Electric Vehicle (PHEV)   
4          Battery Electri

## BUC Implementation - In Memory

In [21]:
def buc(data, dimensions, min_support=0, prefix=()):
    if len(dimensions) == 0:
        count = len(data)
        if count >= min_support:
            result = {tuple(prefix): count}
        else:
            result = {}
    else:
        dim = dimensions[0]
        rest_dims = dimensions[1:]
        dim_values = data[dim].unique()

        result = {}
        for value in dim_values:
            subset = data[data[dim] == value]
            new_prefix = prefix + ((dim, value),)
            subresult = buc(subset, rest_dims, min_support, new_prefix)
            result.update(subresult)

        if len(dim_values) > 1:
            all_prefix = prefix + ((dim, 'ALL'),)
            all_result = buc(data, rest_dims, min_support, all_prefix)
            result.update(all_result)

    return result

In [22]:
dimensions = [column for column in data.columns]
minsup=10000
measures = ['Count']
df=pd.DataFrame(data)
result = buc(df, dimensions, minsup)

In [23]:
result_list = []
for key, value in result.items():
    row_data = {dim: dim_value for dim, dim_value in key}
    row_data[measures[0]] = value
    result_list.append(row_data)

result_df = pd.DataFrame(result_list, columns=dimensions+measures)
result_df.head(10)

,County,City,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Utility,Count
0,King,Seattle,ALL,TESLA,ALL,Battery Electric Vehicle (BEV),ALL,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),11347
1,King,Seattle,ALL,TESLA,ALL,Battery Electric Vehicle (BEV),ALL,ALL,11900
2,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),14556
3,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,ALL,15254
4,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),ALL,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),22829
5,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),ALL,ALL,23961
6,King,Seattle,ALL,ALL,ALL,ALL,Clean Alternative Fuel Vehicle Eligible,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),10955
7,King,Seattle,ALL,ALL,ALL,ALL,Clean Alternative Fuel Vehicle Eligible,ALL,11543
8,King,Seattle,ALL,ALL,ALL,ALL,Eligibility unknown as battery range has not b...,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),14556
9,King,Seattle,ALL,ALL,ALL,ALL,Eligibility unknown as battery range has not b...,ALL,15254


## BUC Implementation - Out-of-Memory

In [46]:
def buc_withpaging(data, dimensions, min_support=0, prefix=(), chunk_size=None):
    def process(data, dimensions, min_support=0, prefix=()):
      if len(dimensions) == 0:
        count = len(data)
        result = {tuple(prefix): count}
      else:
          dim = dimensions[0]
          rest_dims = dimensions[1:]
          dim_values = data[dim].unique()

          result = {}
          for value in dim_values:
              subset = data[data[dim] == value]
              new_prefix = prefix + ((dim, value),)
              subresult = process(subset, rest_dims, min_support, new_prefix)
              result.update(subresult)

          if len(dim_values) > 1:
              all_prefix = prefix + ((dim, 'ALL'),)
              all_result = process(data, rest_dims, min_support, all_prefix)
              result.update(all_result)

      return result

    result = {}
    i = 0
    for chunk in pd.read_csv('Electric_Vehicle_Data.csv', chunksize=chunk_size):
        print(i)
        i = i+1

        chunk_result = process(chunk, dimensions, min_support, prefix)

        for key, value in chunk_result.items():
          if key in result:
              result[key] += value
          else:
              result[key] = value

    result = {k: v for k, v in result.items() if v >= min_support}

    return result

In [48]:
dimensions = [column for column in data.columns]
minsup=10000
measures = ['Count']
prefix = ()
chunk_size = 10000
df=pd.DataFrame(data)
result = buc_withpaging(df, dimensions, minsup, prefix, chunk_size)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [49]:
result_list = []
for key, value in result.items():
    row_data = {dim: dim_value for dim, dim_value in key}
    row_data[measures[0]] = value
    result_list.append(row_data)

result_df = pd.DataFrame(result_list, columns=dimensions+measures)
result_df.head(10)

,County,City,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Utility,Count
0,King,Seattle,ALL,TESLA,ALL,Battery Electric Vehicle (BEV),ALL,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),11347
1,King,Seattle,ALL,TESLA,ALL,Battery Electric Vehicle (BEV),ALL,ALL,11235
2,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),14556
3,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,ALL,14422
4,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),ALL,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),22829
5,King,Seattle,ALL,ALL,ALL,Battery Electric Vehicle (BEV),ALL,ALL,22642
6,King,Seattle,ALL,ALL,ALL,ALL,Clean Alternative Fuel Vehicle Eligible,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),10955
7,King,Seattle,ALL,ALL,ALL,ALL,Clean Alternative Fuel Vehicle Eligible,ALL,10906
8,King,Seattle,ALL,ALL,ALL,ALL,Eligibility unknown as battery range has not b...,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),14556
9,King,Seattle,ALL,ALL,ALL,ALL,Eligibility unknown as battery range has not b...,ALL,14422


## Performance Analysis

### Plot of minsup vs runtime, keeping allotted memory fixed

In [27]:
minsup_list = [1000, 5000, 10000, 20000, 50000]
time_list = []

for minsup in minsup_list:
    start_time = time.time()
    prefix = ()
    chunk_size = 1000
    result = buc_withpaging(df, dimensions, minsup, prefix, chunk_size)
    end_time = time.time()
    time_list.append(end_time - start_time)

plt.plot(minsup_list, time_list)
plt.xlabel('minsup')
plt.ylabel('time')
plt.title('Time vs minsup')
plt.grid(True)
plt.show()

KeyboardInterrupt: 

### Plot of allotted memory vs. runtime, keeping minsup fixed

In [ ]:
chunk_size_list = range(1000,3000,200)
time_list = []

for chunk_size in chunk_size_list:
    start_time = time.time()
    prefix = ()
    minsup = 10000
    result = buc_withpaging(df, dimensions, minsup, prefix, chunk_size)
    end_time = time.time()
    time_list.append(end_time - start_time)

plt.plot(chunk_size_list, time_list)
plt.xlabel('chunk_size')
plt.ylabel('time')
plt.title('Time vs chunk_size')
plt.grid(True)
plt.show()

## Optimization Technique

## BUC - Apriori Pruning

In [24]:
def buc_pruning(data, dimensions, min_support=0, prefix=()):
    if len(dimensions) == 0:
        count = len(data)
        if count >= min_support:
            result = {tuple(prefix): count}
        else:
            result = {}
    else:
        dim = dimensions[0]
        rest_dims = dimensions[1:]
        dim_values = data[dim].unique()

        result = {}
        for value in dim_values:
            subset = data[data[dim] == value]
            count = len(subset)
            # Prune branches where the count is already less than min_support
            if count >= min_support:
                new_prefix = prefix + ((dim, value),)
                subresult = buc_pruning(subset, rest_dims, min_support, new_prefix)
                result.update(subresult)

        # Prune branches where the count is already less than min_support
        if len(dim_values) > 1:
            all_prefix = prefix + ((dim, 'ALL'),)
            all_count = len(data)
            if all_count >= min_support:
                all_result = buc_pruning(data, rest_dims, min_support, all_prefix)
                result.update(all_result)

    return result

In [25]:
dimensions = [column for column in data.columns]
minsup=10000
measures = ['Count']
df=pd.DataFrame(data)
result = buc_pruning(df, dimensions, minsup)

In [26]:
result_list = []
for key, value in result.items():
    row_data = {dim: dim_value for dim, dim_value in key}
    row_data[measures[0]] = value
    result_list.append(row_data)

result_df = pd.DataFrame(result_list, columns=dimensions+measures)
result_df = result_df.sort_values(by=measures[0], ascending=False)
result_df.head(10)

,County,City,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Utility,Count
176,ALL,ALL,ALL,ALL,ALL,ALL,ALL,ALL,181455
162,ALL,ALL,ALL,ALL,ALL,Battery Electric Vehicle (BEV),ALL,ALL,141970
171,ALL,ALL,ALL,ALL,ALL,ALL,Eligibility unknown as battery range has not b...,ALL,94730
158,ALL,ALL,ALL,ALL,ALL,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,ALL,94730
61,King,ALL,ALL,ALL,ALL,ALL,ALL,ALL,94460
133,ALL,ALL,ALL,TESLA,ALL,Battery Electric Vehicle (BEV),ALL,ALL,80816
52,King,ALL,ALL,ALL,ALL,Battery Electric Vehicle (BEV),ALL,ALL,76225
174,ALL,ALL,ALL,ALL,ALL,ALL,ALL,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),67180
167,ALL,ALL,ALL,ALL,ALL,ALL,Clean Alternative Fuel Vehicle Eligible,ALL,66813
60,King,ALL,ALL,ALL,ALL,ALL,ALL,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),62126
